# Better Way 40. 많은 함수를 동시에 실행하려면 코루틴을 고려하자
* 스레드를 사용하는 데 있는 세 가지 문제
  1. **스레드들이 서로 안전하게 동작하도록 조율하려면 특별한 도구가 필요하다.**
    * Better way 38, 39 참조
    * 스레드를 사용하는 코드가 절차적인 싱글 스레드 코드보다 이해하기 어렵고,
    * 복잡성 때문에 시간이 지날수록 스레드 코드를 확장하거나 유지보수하기 어렵다.
  2. **스레드에는 메모리가 많이 필요하다** (스레드 당 8MB 정도)
    * 스레드 수십개 가량 쓰는것은 보통 문제가 되지 않지만, 
    * 사용자가 서버에 보내는 요청, 화면의 픽셀, 시뮬레이션의 입자 등 고유의 활동마다 스레드 하나씩 실행하면... 그래서 스레드 수천개가 동시에 실행해야 한다면 제대로 동작하지 않을 것이다.
  3. **스레드를 시작하는 데에 비용이 많이 든다.**
    * 끊임없이 새 병행 함수를 생성하고 종료하면 스레드를 사용하는 데 드는 부하가 커져서 전체 시스템이 느려진다.

* 스레드 대신 코루틴 coroutine 을 사용해보자.
  * 코루틴을 사용하면 동시에 많은 함수를 실행하는 것처럼 보이게 할 수 있다.
  * 코루틴은 제너레이터의 확장
    * Better Way 16
  * 코루틴의 비용: 함수 호출 (1KB 미만의 메모리만 소비한다)

* 코루틴은 제너레이터를 소비하는 코드에서 send 함수를 사용하여 역으로 제너레이터 함수의 각 yield 표현식에 값을 보낼 수 있게 하는 방법으로 동작
* 제너레이터 함수는 send 함수로 보낸 값을 대응하는 yield 표현식의 결과로 받는다

In [1]:
def my_coroutine():
    while True:
        received = yield
        print('Received:', received)

In [2]:
it = my_coroutine()
next(it) # 코루틴을 준비함

it.send('First')
it.send('Second')

Received: First
Received: Second


* 보충설명
    * 제너레이터를 소비하는 코드 (`def my_coroutine`) 에서 send 함수를 사용 (`it.send`) 하여 역으로 제너레이터 함수의 각 yield 표현식에 값을 보낼 수 있게 하는 방법으로 동작
    * 제너레이터 함수 (`def my_coroutine`) 는 send 함수로 보낸 값 (`First`, `Second`) 을 대응하는 yield 표현식의 결과 (`received = yield`) 로 받는다

* 제너레이터가 첫 번째 `yield` 표현식으로 전진해서 첫 번째 `send`의 값을 받을 수 있게 하려면 먼저 `next` 를 호출해야 한다.
* `yield` 와 `send` 의 조합은 제너레이터가 외부 입력에 반응하여 다음 번에 다른 값을 얻게 하는 표준 방법이다.
* 예시) 지금까지 보낸 값 중에서 최솟값을 넘겨주는 제너레이터 코루틴 구현
  * 넘길 값이 없는 `yield` 로 외부에서 보낸 초기 최솟값을 받아서 코루틴을 준비
  * 이후 제너레이터가 반복적으로 다음 값을 받으면서 새 최솟값을 넘겨줌

In [3]:
def minimize():
    current = yield
    while True:
        value = yield current
        current = min(value, current)

In [4]:
it = minimize()
next(it)
print(it.send(10))
print(it.send(4))
print(it.send(22))
print(it.send(-1))
print(it.send(10))

10
4
4
-1
-1


* 스레드와 코루틴 비교
  * 둘 다 주변 환경에서 받은 입력을 소비하여 결과를 만들어 낼 수 있는 독립적인 함수
  * 스레드와 다르게, 코루틴은 제너레이터 함수의 각 `yield` 표현식에서 멈췄다가 외부에서 `send` 를 호출할때마다 다시 시작
    * 덕분에, 제너레이터를 소비하는 코드에서 코루틴의 각 `yield` 표현식 이후에 원하는 처리를 할 수 있음
      * 제너레이터의 출력 값으로 다른 함수를 호출하거나, 자료구조를 수정하거나...
      
* 다른 제너레이터 함수들을 `yield` 표현식 이전까지 전진시킬 수 있다는 점이 중요
  * 많은 별개의 코루틴을 사용하면 파이썬 스레드의 병행 동작을 흉내내며 동시에 실행하는 것처럼 보인다

### 생명 게임
* 게임 규칙
  * 임의의 크기인 2차원 그리드
  * 그리드의 각 셀은 살아 있거나 (alive), 죽어 있거나 (empty) 둘 중 하나

In [5]:
ALIVE = '*'
EMPTY = '-'

* 게임 규칙
  * 한 번에 한 틱씩 진행
    * 틱이 발생할 때 마다 각 셀은 이웃한 셀 8개 중 몇개가 살아있는지 센다
    * 살아남은 이웃의 개수에 근거하여 계속 살아갈지, 죽을지, 되살아날지를 결정
      * 너무 적어도 죽고, 너무 많아도 죽고, 적절한 수준일때만 살 수 있다...

```
  0   |   1   |   2   |   3   |   4   
----- | ----- | ----- | ----- | -----
-*--- | --*-- | --**- | --*-- | -----
--**- | --**- | -*--- | -*--- | -**--
---*- | --**- | --**- | --*-- | -----
----- | ----- | ----- | ----- | -----
```

* 이웃 셀의 상태를 알아내는 방법이 필요
* Query 클래스는 제너레이터 코루틴이 주변 환경에 정보를 요청할 방법을 제공하는 클래스

In [6]:
from collections import namedtuple

Query = namedtuple('Query', ('y', 'x'))

* 코루틴은 각 이웃별로 Query 를 넘겨준다.
  * yield 표현식의 결과는 ALIVE 나 EMPTY
* `count_neighbors` 코루틴
  * 이웃의 상태를 확인하고 살아있는 이웃의 수를 반환

In [7]:
def count_neighbors(y, x):
    n_ = yield Query(y+1, x+0) # 북
    ne = yield Query(y+1, x+1) # 북동
    e_ = yield Query(y+0, x+1) # 동
    se = yield Query(y-1, x+1) # 남동
    s_ = yield Query(y-1, x+0) # 남
    sw = yield Query(y-1, x-1) # 남서
    w_ = yield Query(y+0, x-1) # 서
    nw = yield Query(y+1, x-1) # 북서
    
    neighbor_states = [n_, ne, e_, se, s_, sw, w_, nw]
    
    count = 0
    for state in neighbor_states:
        if state == ALIVE:
            count += 1
    return count

* 테스트
  * 최종 개수는 return 문으로 끝날 때 일어나는 `StopIteration` 예외로 반환됨

In [8]:
it = count_neighbors(10, 5)
q1 = next(it)                  # Get the first query
print('First yield: ', q1)
q2 = it.send(ALIVE)            # Send q1 state, get q2
print('Second yield:', q2)
q3 = it.send(ALIVE)            # Send q2 state, get q3
print('...')
q4 = it.send(EMPTY)
q5 = it.send(EMPTY)
q6 = it.send(EMPTY)
q7 = it.send(EMPTY)
q8 = it.send(EMPTY)
try:
    it.send(EMPTY)     # Send q8 state, retrieve count
except StopIteration as e:
    print('Count: ', e.value)  # Value from return statement

First yield:  Query(y=11, x=5)
Second yield: Query(y=11, x=6)
...
Count:  2


* `step_cell` 코루틴
  * `count_neighbors` 에서 찾은 이웃 카운터에 대응하도록 새로운 상태로 변할 것을 알려주는 기능 구현
    * `count_neighbors` 를 실행하여 주변 셀을 조사하고,
    * 게임 로직에 의해 다음 clock tick 에 어떤 상태여야 하는지 결정하고,
    * `Transition`객체로 셀의 상태 변화를 알린다.

In [9]:
Transition = namedtuple('Transition', ('y', 'x', 'state'))

In [10]:
def step_cell(y, x):
    state = yield Query(y, x)
    neighbors = yield from count_neighbors(y, x)
    next_state = game_logic(state, neighbors)
    yield Transition(y, x, next_state)

In [11]:
def game_logic(state, neighbors):
    if state == ALIVE:
        if neighbors < 2:
            return EMPTY          # Die: Too few
        elif neighbors > 3:
            return EMPTY          # Die: Too many
    else:
        if neighbors == 3:
            return ALIVE          # Regenerate
    return state

* `count_neighbors` 를 호출할 때 `yield from` 을 사용한다.
  * 제너레이터 코루틴을 조합하여 더 작은 기능을 재사용하고 간단한 코루틴을 조립하기 더 편함
  * `count_neighbors` 가 모두 소진되면 return 문으로 반환하는 최종값은 `yield from` 을 통해 `step_cell` 에 전달된다.

* 테스트

In [12]:
it = step_cell(10, 5)
q0 = next(it)  # 초기 위치 쿼리
print('Me         ', q0)
q1 = it.send(ALIVE)  # 내 상태를 전달하고 이웃 쿼리를 받음
print('Q1:        ', q1)
q2 = it.send(EMPTY)
print('Q2:        ', q2)
q3 = it.send(ALIVE)
print('Q3:        ', q3)
q4 = it.send(EMPTY)
print('Q4:        ', q4)
q5 = it.send(EMPTY)
print('Q5:        ', q5)
q6 = it.send(EMPTY)
print('Q6:        ', q6)
q7 = it.send(EMPTY)
print('Q7:        ', q7)
q8 = it.send(ALIVE)
print('Q8:        ', q8)
t1 = it.send(EMPTY)  # q8 상태를 전달하고 게임 상태를 받음
print('Outcome:   ', t1)

Me          Query(y=10, x=5)
Q1:         Query(y=11, x=5)
Q2:         Query(y=11, x=6)
Q3:         Query(y=10, x=6)
Q4:         Query(y=9, x=6)
Q5:         Query(y=9, x=5)
Q6:         Query(y=9, x=4)
Q7:         Query(y=10, x=4)
Q8:         Query(y=11, x=4)
Outcome:    Transition(y=10, x=5, state='*')


* 그리드의 모든 셀에 로직을 확장해보자
  * `simulate` 코루틴

In [13]:
TICK = object()

def simulate(height, width):
    while True:
        for y in range(height):
            for x in range(width):
                yield from step_cell(y, x)
        yield TICK

* `simulate` 코루틴
  * 외부 환경과 완전히 분리되어 있음
  * 그리드를 파이썬 객체로 표현할 방법과, `Query`, `Transitio`, `TICK` 을 외부에서 처리할 방법, 초기 상태를 얻을 방법은 아직 정의하지 않음
  * 하지만 로직은 명확함

* 코루틴을 이용하면 처리할 로직에 집중할 수 있다.
  * 원하는 작업을 구현한 부분에서 환경에 해당하는 코드의 명령어를 분리할 수 있다.
  * 코루틴을 병렬로 동작하는 것처럼 실행할 수 있다.
  * 시간이 지나서도 코루틴을 변경하지 않고 이러한 명령어 이후에 오는 구현을 개선할 수 있다.

* 그리드를 구현해보자

In [14]:
class Grid(object):
    def __init__(self, height, width):
        self.height = height
        self.width = width
        self.rows = []
        
        for _ in range(self.height):
            self.rows.append([EMPTY] * self.width)

    def __str__(self):
        output = ''
        for row in self.rows:
            for cell in row:
                output += cell
            output += '\n'
        return output

    def query(self, y, x):
        return self.rows[y % self.height][x % self.width] # 범위를 벗어난 좌표는 반대쪽으로

    def assign(self, y, x, state):
        self.rows[y % self.height][x % self.width] = state

* `simulate` 에서 나온 값을 해석해서, 그 내부에서 사용하도록 만드는 코루틴을 정의
  * 코루틴에서 나온 명령어를 주변 환경과의 상호 작용으로 변환한다
  * 그리드 전체를 한 단계 진전시킨 후, 다음 상태를 담은 새로운 그리드를 반환

In [15]:
def live_a_generation(grid, sim):
    progeny = Grid(grid.height, grid.width)
    item = next(sim)
    
    while item is not TICK:
        if isinstance(item, Query):
            state = grid.query(item.y, item.x)
            item = sim.send(state)
        else:  # Must be a Transition
            progeny.assign(item.y, item.x, item.state)
            item = next(sim)
    
    return progeny

* 그리드를 생성하고 초기 상태를 설정해서 글라이더 glider 라는 고전적인 모양을 만들어보자
![Other classical constructions](http://pentadecathlon.com/lifeNews/2006/06/collisions/trivial/trivial.ltbl?bits=8&columns=6)

In [16]:
grid = Grid(5, 9)
grid.assign(0, 3, ALIVE)
grid.assign(1, 4, ALIVE)
grid.assign(2, 2, ALIVE)
grid.assign(2, 3, ALIVE)
grid.assign(2, 4, ALIVE)
print(grid)

---*-----
----*----
--***----
---------
---------



* 이제 이 그리드를 한번에 한 세대 씩 진행해보자

In [17]:
class ColumnPrinter(object):
    def __init__(self):
        self.columns = []

    def append(self, data):
        self.columns.append(data)

    def __str__(self):
        row_count = 1
        for data in self.columns:
            row_count = max(row_count, len(data.splitlines()) + 1)
        rows = [''] * row_count
        for j in range(row_count):
            for i, data in enumerate(self.columns):
                line = data.splitlines()[max(0, j - 1)]
                if j == 0:
                    padding = ' ' * (len(line) // 2)
                    rows[j] += padding + str(i) + padding
                else:
                    rows[j] += line
                if (i + 1) < len(self.columns):
                    rows[j] += ' | '
        return '\n'.join(rows)

In [18]:
colums = ColumnPrinter()
sim = simulate(grid.height, grid.width)
for i in range(5):
    colums.append(str(grid))
    grid = live_a_generation(grid, sim)

print(colums)

    0     |     1     |     2     |     3     |     4    
---*----- | --------- | --------- | --------- | ---------
----*---- | --*-*---- | ----*---- | ---*----- | ----*----
--***---- | ---**---- | --*-*---- | ----**--- | -----*---
--------- | ---*----- | ---**---- | ---**---- | ---***---
--------- | --------- | --------- | --------- | ---------


* 이런 식으로 코루틴을 쓰면...
  * 주변 코드를 업데이트하지 않고도 `game_logic` 함수를 변경할 수 있다
    * `Query`, `Transition`, `TICK` 을 그대로 두고 규칙을 변경하거나 더 큰 영향력 있는 요소를 넣을 수도 있다
  * 코루틴의 중요한 설계 원칙 중 하나: 관심 영역의 분리
    * 를 어떻게 가능하게 하는지 보여준다

### 파이썬 2의 코루틴
* 파이썬 2에는 `yield from` 이 없다

In [19]:
def delegated():
    yield 1
    yield 2

def composed():
    yield 'A'
    for value in delegated():  # yield from in Python 3
        yield value
    yield 'B'

print(list(composed()))

['A', 1, 2, 'B']


* 파이썬 2의 제너레이터에는 return 문을 지원하지 않는다
  * `try/except/finally` 블록과 올바르게 상호작용하는 동작을 구현하려면 직접 예외 타입을 정의한 후 값을 반환하고 싶을 때 해당 예외를 일으켜야 한다

In [20]:
class MyReturn(Exception):
    def __init__(self, value):
        self.value = value

def delegated():
    yield 1
    raise MyReturn(2)  # return 2 in Python 3
    yield 'Not reached'

def composed():
    try:
        for value in delegated():
            yield value
    except MyReturn as e:
        output = e.value
    yield output * 4

print(list(composed()))

[1, 8]


### 핵심 정리
* 코루틴은 함수 수만 개를 마치 동시에 실행하는 것처럼 실행하는 효과적인 방법을 제공한다.
* 제너레이터 안에서 `yield` 표현식의 값은 외부 코드에서 제너레이터의 `send` 메서드에 전달한 값이다.
* 코루틴은 프로그램의 핵심 로직을 주변 환경과 상호작용하는 코드로부터 분리할 수 있는 강력한 도구이다.
* 파이썬 2는 `yield from` 문법과 제너레이터에서 값을 반환하는 기능을 지원하지 않는다.